In [17]:
import bigml.api

In [18]:
from pandas import read_csv

### Import the project

In [20]:
#We already created the docker file
import bigml.api
from bigml.api import BigML
api= BigML (project='project/5d94a52e5a213962e2000331')

### Create the source

In [21]:
source = api.create_source('fulltrain_dataframe.csv')

In [22]:
api.ok(source)

True

In [23]:
#Create the dataset
dataset_1 = api.create_dataset(source)

In [24]:
api.ok(dataset_1)

True

In [25]:
#Split the trainfull in train80/val20
train_dataset = api.create_dataset(
    dataset_1, {"name": "Train80",
                     "sample_rate": 0.8, "seed": "my seed"})
val = api.create_dataset(
    dataset_1, {"name": "Train20",
                     "sample_rate": 0.8, "seed": "my seed",
                     "out_of_bag": True})

In [26]:
api.ok(train_dataset)

True

In [27]:
api.ok(val)

True

In [32]:
api.download_dataset(train_dataset, filename='train_dataset.csv')

'train_dataset.csv'

In [33]:
api.download_dataset(val, filename='val.csv')

'val.csv'

In [10]:
#Launch the model ensemble
model = api.create_ensemble(train_dataset, {"objective_field":"SeriousDlqin2yrs"})

In [11]:
api.ok(model)

True

In [12]:
#Launch the batch prediction on the validation set
batch_prediction = api.create_batch_prediction(model, val, {
    "all_fields": True, "confidence": True, "probabilities": True})

In [13]:
api.ok(batch_prediction)

True

In [14]:
#Download the new prediction 
api.download_batch_prediction(batch_prediction,
    filename='../handson-ml2/prediction_credit.csv')

'../handson-ml2/prediction_credit.csv'

In [15]:
#Upload the modified test
test_source = api.create_source('test_dataframe.csv')

In [16]:
api.ok(test_source)

True

In [17]:
#Create the dataset test
dataset_2 = api.create_dataset(test_source)

In [18]:
api.ok(dataset_2)

True

In [19]:
#Model ensemble on the test set
model = api.create_ensemble(dataset_1, {"objective_field":"SeriousDlqin2yrs"})

In [20]:
api.ok(model)

True

In [21]:
#Batch prediction on the test set/add column probabilities
batch_prediction = api.create_batch_prediction(model, dataset_2, {"all_fields": True, "confidence": True, "probabilities": True})

In [22]:
api.ok(batch_prediction)

True

In [23]:
#Evaluation on the test
evaluation = api.create_evaluation(model, dataset_1)
api.ok(evaluation)

True

In [24]:
 api.pprint(evaluation['object']['result']['model']['average_area_under_roc_curve'])

0.88198


In [25]:
api.download_batch_prediction(batch_prediction,
    filename='../Credit_p/prediction_credit_final.csv')

'../Credit_p/prediction_credit_final.csv'

In [26]:
# Import the new prediction and remove the columns/replace Id and Probability
import pandas as pd
df = pd.read_csv('prediction_credit_final.csv')
df.drop(df.columns[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]], axis=1, inplace=True)
df.rename(columns={'field1': 'Id', '1 probability': 'Probability'}, inplace=True)
df.to_csv('prediction_credit_final_clean.csv', index=False)

In [2]:
import kaggle
kaggle.api.competition_submit("prediction_credit_final_clean.csv", "test", "House Prices: Advanced Regression Techniques")

ApiException: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'Date': 'Sun, 24 Nov 2019 10:57:08 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Vary': 'Accept-Encoding', 'Access-Control-Allow-Credentials': 'true', 'Turbolinks-Location': 'https://www.kaggle.com/api/v1/competitions/House%20Prices:%20Advanced%20Regression%20Techniques/submissions/url/1735692/1571911474', 'X-Kaggle-MillisecondsElapsed': '5', 'X-Kaggle-RequestId': '8d59ff6ca807dec4b167a37ac120dc38', 'X-Kaggle-ApiVersion': '1.5.6', 'X-Frame-Options': 'SAMEORIGIN', 'Strict-Transport-Security': 'max-age=31536000', 'Content-Security-Policy': "object-src 'none'; base-uri 'none'; report-uri /csp/report", 'Content-Security-Policy-Report-Only': "object-src 'none'; script-src 'nonce-5abtPG2tjb/TVA/p7fPYrg==' 'report-sample' 'unsafe-inline' 'unsafe-eval' 'strict-dynamic' https: http:; frame-src 'self' https://www.kaggleusercontent.com https://www.youtube.com/embed/ https://polygraph-cool.git